## Data Cleaning

**Analyse data**

The next thing that we should do is clean the data stored in BA_reviews.csv file to remove any unnecessary text from each of the rows.

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [14]:
df=pd.read_csv('BA_reviews.csv', index_col=0)
df.head()

,reviews,stars,date
0,✅ Trip Verified | My family and I have flown ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,9th July 2023
1,✅ Trip Verified | This has been by far the wo...,4,9th July 2023
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,8th July 2023
3,Not Verified | BA is not treating its premium ...,2,6th July 2023
4,✅ Trip Verified | 24 hours before our departu...,4,5th July 2023


creating a column if the user trip if verified ✅ or not 

In [15]:
df['Verified']=df['reviews'].str.contains("Trip Verified")
df['Verified']

0        True
1        True
2        True
3       False
4        True
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: Verified, Length: 3500, dtype: bool

### Cleaning reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [23]:
import nltk
import re
nltk.download('stopwords')
stemmer=nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text=str(text).lower()
    text=re.sub('\s+',' ',text) #remove extra whitespaces
    text=re.sub('RT|cc',' ',text) #remove RT and cc
    text=re.sub('[%s]'% re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""),' ',text) #remove punctuations
    text=re.sub('#\S+','',text) #remove hashtags
    text=re.sub('http\S+\s*', ' ',text) #remove URLs
    text=re.sub(r'[^\x00-\x7f]',r' ',text)
    text=re.sub('@\S+','',text) #remove mentions
    text=[word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text=[stemmer.stem(word) for word in text.split(" ")]
    text=" ".join(text)
    return text
    
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mutho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
df['reviews']=df['reviews'].apply(clean)

In [25]:
df.head()

,reviews,stars,date,Verified
0,trip verifi famili flown most british airway ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,9th July 2023,True
1,trip verifi far worst servic plane obvious fl...,4,9th July 2023,True
2,trip verifi nov 2022 book paid return journey...,2,8th July 2023,True
3,verifi ba treat premium economi passeng respec...,2,6th July 2023,False
4,trip verifi 24 hour departur ba059 cape town ...,4,5th July 2023,True


### Cleaning date

In [26]:
df.dtypes

reviews     object
stars       object
date        object
Verified      bool
dtype: object

In [27]:
# convert the date to datetime format
df['date'] = pd.to_datetime(df['date'])

In [28]:
df['date'].head()

0   2023-07-09
1   2023-07-09
2   2023-07-08
3   2023-07-06
4   2023-07-05
Name: date, dtype: datetime64[ns]

### Cleaning  stars

In [29]:
#check for unique values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '4', '2', '1', '3', '10', '7',
       '9', '5', '8', '6', 'None'], dtype=object)

In [30]:
# remove the \t and \n from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [31]:
df.stars.value_counts()

1       797
2       400
3       393
8       348
7       299
10      299
9       292
5       253
4       239
6       175
None      5
Name: stars, dtype: int64

There are 5 rows having values "None" in the ratings. We will drop all these 5 rows.

In [32]:
# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [34]:
#check the unique values again
df.stars.unique()

array(['5', '4', '2', '1', '3', '10', '7', '9', '8', '6'], dtype=object)

In [36]:
df.head()

,reviews,stars,date,Verified
0,trip verifi famili flown most british airway ...,5,2023-07-09,True
1,trip verifi far worst servic plane obvious fl...,4,2023-07-09,True
2,trip verifi nov 2022 book paid return journey...,2,2023-07-08,True
3,verifi ba treat premium economi passeng respec...,2,2023-07-06,False
4,trip verifi 24 hour departur ba059 cape town ...,4,2023-07-05,True


In [37]:
df.shape

(3495, 4)

In [35]:
df.isnull().sum()

reviews     0
stars       0
date        0
Verified    0
dtype: int64

In [38]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,Verified
0,trip verifi famili flown most british airway ...,5,2023-07-09,True
1,trip verifi far worst servic plane obvious fl...,4,2023-07-09,True
2,trip verifi nov 2022 book paid return journey...,2,2023-07-08,True
3,verifi ba treat premium economi passeng respec...,2,2023-07-06,False
4,trip verifi 24 hour departur ba059 cape town ...,4,2023-07-05,True
...,...,...,...,...
3490,ba 213 lhr boston t5 busi check fast effici fl...,7,2014-06-12,False
3491,flew world travel plus first time trip lhr tor...,5,2014-06-12,False
3492,glasgow lhr complet full flight crew amaz desp...,4,2014-06-12,False
3493,lhr cph lhr a320 buss stand outward good bus u...,7,2014-06-12,False


Now our data is all cleaned and ready for data visualization and data analysis.

In [40]:
# export the cleaned data
import os

cwd = os.getcwd()
df.to_csv(cwd + "/cleaned-BA-reviews.csv")